In [57]:
import json
import requests
import openai
import time
import os
import pandas as pd

In [53]:
#import from deidentified_radiology_findings file into a pandas dataframe
import pandas as pd
df = pd.read_csv('cleaned_training.csv')
df


# #remove all columns except for the 'findings' column
df = df[['findings']]
df.head()

# #convert the dataframe to a list
prompts = df.values.tolist()

# #flatten the list
prompts = [item for sublist in prompts for item in sublist]

# # for every entry in the list, add "Is there any abnormal acute lung or pleura findings in the following chest x ray report? \n"
# # to the beginning of the entry and "\n Respond only with "Normal" or "Abnormal" to the end of the entry
for i in range(len(prompts)):
    prompts[i] = "Is there any abnormal acute lung or pleura findings in the following chest x ray report?: '" + str(prompts[i]) + "' Respond only with 'Normal' or 'Abnormal'"


# # prompts = prompts[:5]
prompts


["Is there any abnormal acute lung or pleura findings in the following chest x ray report?: 'tubes and catheters: none. central airways: normal. lungs/pleura/pleural space: interstitial prominence, concerning for edema. there are superimposed patchy areas of airspace opacity bilaterally, possibly related to confluent edema and/or a multifocal infectious/inflammatory process. heart and mediastinum: the heart is enlarged. additional findings: no acute or aggressive osseous changes noted.' Respond only with 'Normal' or 'Abnormal'",
 "Is there any abnormal acute lung or pleura findings in the following chest x ray report?: 'tubes and catheters: none. central airways: normal. lungs: bilateral lower lobe infiltrates on this hypoventilatory exam. right upper lobe infiltrate as well. pleura/pleural space: normal. heart and mediastinum: normal. additional findings: no acute or aggressive osseous changes noted.' Respond only with 'Normal' or 'Abnormal'",
 "Is there any abnormal acute lung or ple

In [27]:


openai.api_key = #REDACTED

# prompts = [
#     "What is the meaning of life?",
#     "How can I become a better writer?",
#     "Tell me a joke.",
#     "What's your favorite book?",
#     "What is the airspeed velocity of an unladen swallow?",
# ]

responses = []

for p in prompts:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=p,
    temperature=0,
    max_tokens=64,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
  )
  responses.append(response.choices[0].text.strip())
  time.sleep(1)  # Pause for 1 second to avoid rate limiting

#print the responses
print(responses)


# start_sequence = "\nA:"
# restart_sequence = "\n\nQ: "

# response = openai.Completion.create(
#   model="gpt-3.5-turbo",
#   prompt=p,
#   temperature=0,
#   max_tokens=100,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0,
#   stop=["\n"]
# )

['Normal', 'Normal', 'Normal', 'Normal', 'Normal']


In [43]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work

openai.api_key = #REDACATED

responses = []

for p in prompts:
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "system", "content": "Respond only with 'Abnormal'."},
          {"role": "user", "content": p},
      ]
  )
  responses.append(response.choices[0].message.content)
  time.sleep(1)  # Pause for 1 second to avoid rate limiting

print(responses)

['Abnormal.', 'Abnormal.', 'Abnormal.', 'Abnormal (low lung volumes)', 'Abnormal.']


In [56]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work

openai.api_key = #REDACATED

responses = []

#create new txt file
f = open("output.txt", "w")

for i in range(len(prompts)):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "Respond only with 'Abnormal'."},
            {"role": "user", "content": prompts[i]},
        ]
    )
    #write the response to the txt file
    f.write(str(i) + ", " + response.choices[0].message.content + "\n")

    #flush the buffer
    f.flush()

    # responses.append(response.choices[0].message.content)
    time.sleep(1)  # Pause for 1 second to avoid rate limiting

f.close()
# print(responses)

In [87]:
#import the output.csv file into a pandas dataframe without the index column
df = pd.read_csv('output.csv')
df

# #import the cleaned_training.csv file into a pandas dataframe
df2 = pd.read_csv('cleaned_training.csv')
df2

# # #set the cxr_abnormal column in df2 to the cxr_abnormal column in df
df2['cxr_abnormal'] = df['cxr_abnormal']

#remove the columns cxr_pneumonia and cxr_consolidation
df2 = df2.drop(columns=['cxr_pneumonia', 'cxr_consolidation'])

#export the dataframe to a csv file
df2.to_csv('ChatGPT_labelled_training.csv', index=False)

